# Importing required packages

In [47]:
from rdkit.Chem import AllChem
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski
from rdkit.ML.Descriptors import MoleculeDescriptors
import pandas as pd
import numpy as np

# Loading the datasets

In [48]:
desc_train=pd.read_csv("train_II.csv")
desc_test=pd.read_csv("test_II.csv")
desc_train.shape

(75383, 2)

Splitting the Compound column given in dataset into Compound and Smiles

In [49]:
desc_train["Compound"]=desc_train["Id"].str.split(";").str[0]
desc_train["Assay_Id"]=desc_train["Id"].str.split(";").str[1]
desc_test["Compound"]= desc_test["x"].str.split(";").str[0]
desc_test["Assay_Id"]= desc_test["x"].str.split(";").str[1]

Extracting smiles for the chemicals

In [50]:
desc_train["Mol"] = [Chem.MolFromSmiles(x) for x in desc_train['Compound']]
desc_test["Mol"] =  [Chem.MolFromSmiles(x) for x in desc_test['Compound']]

[00:06:27] Explicit valence for atom # 1 Si, 8, is greater than permitted
[00:06:28] Explicit valence for atom # 1 Si, 8, is greater than permitted
[00:06:29] Explicit valence for atom # 1 Si, 8, is greater than permitted
[00:06:30] Explicit valence for atom # 1 Si, 8, is greater than permitted
[00:06:31] Explicit valence for atom # 1 Si, 8, is greater than permitted
[00:06:32] Explicit valence for atom # 1 Si, 8, is greater than permitted


Dropping the null columns and resetting the index

In [51]:
desc_train=desc_train.dropna()
desc_test=desc_test.dropna()
desc_train=desc_train.reset_index()
desc_train.drop(['index'], axis=1, inplace=True)

Function for extracting the descriptor names and values for them

In [53]:
def desc(Mol):
    desc_calc = MoleculeDescriptors.MolecularDescriptorCalculator([x[0] for x in Descriptors._descList])
    desc_names = desc_calc.GetDescriptorNames() 
    Mol_desc =[]
    for mol in Mol:
        descriptors = desc_calc.CalcDescriptors(mol)
        Mol_desc.append(descriptors)
    return Mol_desc,desc_names

Calling the descriptors function for train and test files

In [54]:
desc_train1,desc_names_train1 = desc(desc_train["Mol"])
desc_test1, desc_names_test1 =  desc(desc_test["Mol"])

In [55]:
desc_test1 = pd.DataFrame(desc_test1, columns=desc_names_test1)
desc_train1 = pd.DataFrame(desc_train1, columns=desc_names_train1)

In [56]:
desc_test1["Assay_Id2"] = desc_test["Assay_Id"]
desc_test2=pd.concat([desc_test1,desc_test],axis=1)
desc_train1["Assay_Id2"] = desc_train["Assay_Id"]
desc_train2=pd.concat([desc_train1,desc_train],axis=1)

In [46]:
desc_test2

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,Assay_Id2,x,Compound,Assay_Id,Mol
0,9.626968,0.025579,9.626968,0.025579,0.624614,164.248,148.120,164.120115,66,0,...,0,0,0,0,0,1682,CC1=CC(=C(C=C1)C(C)(C)C)O;1682,CC1=CC(=C(C=C1)C(C)(C)C)O,1682,<rdkit.Chem.rdchem.Mol object at 0x7fd79bea0350>
1,12.473362,-4.605249,12.473362,0.061165,0.617511,431.452,414.316,431.056940,152,0,...,0,0,0,0,1,1656,CCS(=O)(=O)C1=C(N=CC=C1)S(=O)(=O)NC(=O)NC2=NC(...,CCS(=O)(=O)C1=C(N=CC=C1)S(=O)(=O)NC(=O)NC2=NC(...,1656,<rdkit.Chem.rdchem.Mol object at 0x7fd79bea0430>
2,14.627193,-4.140552,14.627193,0.064351,0.224134,696.264,655.944,695.250845,254,0,...,0,0,0,0,0,36,CC1=NC2=CC=CC=C2N1C3CC4CCC(C3)N4CCC5(CCN(CC5)C...,CC1=NC2=CC=CC=C2N1C3CC4CCC(C3)N4CCC5(CCN(CC5)C...,36,<rdkit.Chem.rdchem.Mol object at 0x7fd79bea03c0>
3,10.420833,-3.973958,10.420833,0.000000,0.393203,201.244,197.212,200.949810,56,0,...,0,0,0,0,0,1850,CC1=CC(=O)[N-]S(=O)(=O)O1.[K+];1850,CC1=CC(=O)[N-]S(=O)(=O)O1.[K+],1850,<rdkit.Chem.rdchem.Mol object at 0x7fd79bea04a0>
4,12.865865,-0.601027,12.865865,0.094949,0.639062,418.574,380.270,418.271924,168,0,...,0,0,0,0,0,30,CCC(C)(C)C(=O)O[C@H]1C[C@H](C=C2[C@H]1[C@H]([C...,CCC(C)(C)C(=O)O[C@H]1C[C@H](C=C2[C@H]1[C@H]([C...,30,<rdkit.Chem.rdchem.Mol object at 0x7fd79bea0510>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10989,12.851673,-4.443395,12.851673,0.048352,0.391778,408.376,389.224,408.129692,154,0,...,0,0,0,0,0,38,CC(=NOCC1=CC=CC=C1C(=NOC)C(=O)OC)C2=CC(=CC=C2)...,CC(=NOCC1=CC=CC=C1C(=NOC)C(=O)OC)C2=CC(=CC=C2)...,38,<rdkit.Chem.rdchem.Mol object at 0x7fd79c001660>
10990,11.617073,-0.992448,11.617073,0.147894,0.635519,320.344,304.216,320.104859,120,0,...,0,0,0,0,0,34,C1=CC=C(C(=C1)C(C2=CC=C(C=C2)O)C3=CC=C(C=C3)O)...,C1=CC=C(C(=C1)C(C2=CC=C(C=C2)O)C3=CC=C(C=C3)O)...,34,<rdkit.Chem.rdchem.Mol object at 0x7fd79c0016d0>
10991,12.406479,-4.356493,12.406479,0.049357,0.705120,381.370,366.250,381.074304,138,0,...,0,0,0,0,1,1640,CC1=NC(=NC(=N1)OC)NC(=O)NS(=O)(=O)C2=CC=CC=C2C...,CC1=NC(=NC(=N1)OC)NC(=O)NS(=O)(=O)C2=CC=CC=C2C...,1640,<rdkit.Chem.rdchem.Mol object at 0x7fd79c001740>
10992,11.026605,-3.480301,11.026605,0.167292,0.540194,220.976,213.920,219.945901,66,0,...,0,0,0,0,0,28,COP(=O)(OC)OC=C(Cl)Cl;28,COP(=O)(OC)OC=C(Cl)Cl,28,<rdkit.Chem.rdchem.Mol object at 0x7fd79c0017b0>


In [35]:
desc_test2=pd.concat([desc_test1,desc_test])

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,Assay_Id
0,9.626968,0.025579,9.626968,0.025579,0.624614,164.248,148.120,164.120115,66,0,...,0,0,0,0,0,0,0,0,0,1682
1,12.473362,-4.605249,12.473362,0.061165,0.617511,431.452,414.316,431.056940,152,0,...,1,1,0,0,0,0,0,0,1,1656
2,14.627193,-4.140552,14.627193,0.064351,0.224134,696.264,655.944,695.250845,254,0,...,1,0,0,0,0,0,0,0,0,36
3,10.420833,-3.973958,10.420833,0.000000,0.393203,201.244,197.212,200.949810,56,0,...,0,0,0,0,0,0,0,0,0,1850
4,12.865865,-0.601027,12.865865,0.094949,0.639062,418.574,380.270,418.271924,168,0,...,0,0,0,0,0,0,0,0,0,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10989,12.851673,-4.443395,12.851673,0.048352,0.391778,408.376,389.224,408.129692,154,0,...,0,0,0,0,0,0,0,0,0,38
10990,11.617073,-0.992448,11.617073,0.147894,0.635519,320.344,304.216,320.104859,120,0,...,0,0,0,0,0,0,0,0,0,34
10991,12.406479,-4.356493,12.406479,0.049357,0.705120,381.370,366.250,381.074304,138,0,...,1,0,0,0,0,0,0,0,1,1640
10992,11.026605,-3.480301,11.026605,0.167292,0.540194,220.976,213.920,219.945901,66,0,...,0,0,0,0,0,0,0,0,0,28


Creating new dataframe for descriptor values and adding the smiles and id values to that dataframe

Exproting the datasets for loading them into Main file

In [31]:
desc_train2.to_csv("train_I.csv", index=False)
desctest2.to_csv("test_I.csv", index=False)
train1

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,Id2,Id1,Smiles,Id,Expected
0,9.316200,-1.533785,9.316200,0.150485,0.794714,317.599,306.511,315.982463,100,0,...,0,0,0,0,0,1644,C1=CC(=CC=C1C(C2=CC=C(C=C2)O)C(Cl)(Cl)Cl)O,<rdkit.Chem.rdchem.Mol object at 0x7fe02dbc1eb0>,C1=CC(=CC=C1C(C2=CC=C(C=C2)O)C(Cl)(Cl)Cl)O;1644,2
1,10.532611,0.333788,10.532611,0.333788,0.516641,156.269,136.109,156.151415,66,0,...,0,0,0,4,0,2451,CCCCCCCCC(=O)C,<rdkit.Chem.rdchem.Mol object at 0x7fe02dbc1c10>,CCCCCCCCC(=O)C;2451,2
2,2.433032,0.000000,2.433032,0.000000,0.251327,362.086,313.702,361.347528,148,0,...,0,0,0,12,0,1384,CCCCCCCCCC[N+](C)(C)CCCCCCCCCC.[Cl-],<rdkit.Chem.rdchem.Mol object at 0x7fe02dbc1a50>,CCCCCCCCCC[N+](C)(C)CCCCCCCCCC.[Cl-];1384,2
3,10.355080,-0.613825,10.355080,0.282361,0.487998,255.665,245.585,255.052302,90,0,...,0,0,0,0,0,16,C1CN(C(=N1)N[N+](=O)[O-])CC2=CN=C(C=C2)Cl,<rdkit.Chem.rdchem.Mol object at 0x7fe02dbc1c80>,C1CN(C(=N1)N[N+](=O)[O-])CC2=CN=C(C=C2)Cl;16,2
4,0.000000,0.000000,0.000000,0.000000,0.237972,149.894,149.894,149.894242,8,0,...,0,0,0,0,0,1856,[Na+].[I-],<rdkit.Chem.rdchem.Mol object at 0x7fe02dbc1d60>,[Na+].[I-];1856,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75372,11.460021,-3.868472,11.460021,0.053611,0.712426,230.245,220.165,230.036128,82,0,...,0,0,0,0,0,33,COC(=O)NS(=O)(=O)C1=CC=C(C=C1)N,<rdkit.Chem.rdchem.Mol object at 0x7fe09b2c1510>,COC(=O)NS(=O)(=O)C1=CC=C(C=C1)N;33,2
75373,5.928972,-2.841623,5.928972,0.082346,0.720533,313.747,296.611,313.041677,104,0,...,0,0,0,0,0,1632,CCOP(=S)(OCC)OC1=NN(C(=N1)Cl)C(C)C,<rdkit.Chem.rdchem.Mol object at 0x7fe09b2c1580>,CCOP(=S)(OCC)OC1=NN(C(=N1)Cl)C(C)C;1632,1
75374,4.975926,0.848333,4.975926,0.848333,0.596343,167.258,162.218,166.986341,50,0,...,1,0,0,0,0,1373,C1=CC=C2C(=C1)NC(=S)S2,<rdkit.Chem.rdchem.Mol object at 0x7fe09b2c15f0>,C1=CC=C2C(=C1)NC(=S)S2;1373,1
75375,10.241948,0.324028,10.241948,0.324028,0.519485,128.215,112.087,128.120115,54,0,...,0,0,0,0,0,2,CCCCC(CC)C=O,<rdkit.Chem.rdchem.Mol object at 0x7fe09b2c1660>,CCCCC(CC)C=O;2,2
